In [75]:
import pandas as pd
import os

# keyboards_df = pd.read_csv("keyboards_clean2.csv")
monitors_df = pd.read_csv(os.path.join("data","monitors_clean2.csv"))
# mice_df = pd.read_csv("mice_clean2.csv")
# brands_df = pd.read_csv("brands.csv")

# # Get all unique brands from product CSVs
# product_brands = []
# for df in [keyboards_df, monitors_df, mice_df]:
#     product_brands.extend(df["Brand"].tolist())

# product_brands_unique = set(product_brands)
# existing_brands = set(brands_df["brand_name"].tolist())

# # Find brands that are in products but NOT in brands.csv
# missing_brands = product_brands_unique - existing_brands

# print("Missing brands from brands.csv:")
# for brand in sorted(missing_brands):
#     print(f"{brand}")

# print(f"\nTotal missing: {len(missing_brands)}")
# print(f"Total unique product brands: {len(product_brands_unique)}")
# print(f"Total brands in brands.csv: {len(existing_brands)}")

# monitors_df = monitors_df.drop('Price', axis=1)
monitors_df.to_csv(os.path.join("data", "monitors_clean2.csv"), index=False)

In [76]:
def validate_monitor_data(df):
    """
    Validate monitor CSV data against SQL schema types.
    Returns a report of problematic values.
    """
    
    # Define expected types for each column
    schema_mapping = {
        # Physical specifications
        'Size (inch)': ('float', 'DECIMAL(4, 1)'),
        'Curve Radius': ('str', 'VARCHAR(20)'),
        'Wall Mount': ('str', 'VARCHAR(50)'),
        'Borders Size (cm)': ('float', 'DECIMAL(4, 2)'),
        
        # Performance ratings
        'Brightness': ('float', 'DECIMAL(3, 1)'),
        'Response Time': ('float', 'DECIMAL(3, 1)'),
        'HDR Picture': ('float', 'DECIMAL(3, 1)'),
        'SDR Picture': ('float', 'DECIMAL(3, 1)'),
        'Color Accuracy': ('float', 'DECIMAL(3, 1)'),
        
        # Display technology
        'Pixel Type': ('str', 'VARCHAR(20)'),
        'Subpixel Layout': ('str', 'VARCHAR(15)'),
        'Backlight': ('str', 'VARCHAR(50)'),
        'Color Depth (Bit)': ('int', 'INTEGER'),
        
        # Contrast
        'Native Contrast': ('float', 'DECIMAL(8, 1)'),
        'Contrast With Local Dimming': ('float', 'DECIMAL(8, 1)'),
        'Local Dimming': ('bool', 'BOOLEAN'),
        
        # Brightness measurements
        'SDR Real Scene (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        'SDR Peak 100% Window (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        'SDR Sustained 100% Window (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        'HDR Real Scene (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        'HDR Peak 100% Window (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        'HDR Sustained 100% Window (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        'Minimum Brightness (cd/m2)': ('float', 'DECIMAL(6, 2)'),
        
        # Viewing angles
        'Color Washout From Left (degrees)': ('int', 'INTEGER'),
        'Color Washout From Right (degrees)': ('int', 'INTEGER'),
        'Color Shift From Left (degrees)': ('int', 'INTEGER'),
        'Color Shift From Right (degrees)': ('int', 'INTEGER'),
        'Brightness Loss From Left (degrees)': ('int', 'INTEGER'),
        'Brightness Loss From Right (degrees)': ('int', 'INTEGER'),
        'Black Level Raise From Left (degrees)': ('int', 'INTEGER'),
        'Black Level Raise From Right (degrees)': ('int', 'INTEGER'),
        
        # Color accuracy
        'Black Uniformity Native (Std Dev)': ('float', 'DECIMAL(6, 3)'),
        'White Balance (dE)': ('float', 'DECIMAL(4, 2)'),
        
        # Refresh rate
        'Native Refresh Rate (Hz)': ('int', 'INTEGER'),
        'Max Refresh Rate (Hz)': ('int', 'INTEGER'),
        'Native Resolution': ('str', 'VARCHAR(20)'),
        'Aspect Ratio': ('str', 'VARCHAR(10)'),
        'Flicker-Free': ('bool', 'BOOLEAN'),
        
        # Connectivity
        'Max Refresh Rate Over HDMI (Hz)': ('int', 'INTEGER'),
        'DisplayPort': ('str', 'VARCHAR(50)'),
        'HDMI': ('str', 'VARCHAR(50)'),
        'USB-C Ports': ('int', 'INTEGER'),
    }
    
    problems = []
    
    for col_name, (dtype, sql_type) in schema_mapping.items():
        if col_name not in df.columns:
            problems.append(f"❌ Column '{col_name}' not found in CSV")
            continue
        
        print(f"\n{'='*60}")
        print(f"Validating: {col_name} (Expected: {sql_type})")
        print(f"{'='*60}")
        
        col_data = df[col_name]
        
        # Check for problematic values
        if dtype == 'int':
            # Try to convert to int
            for idx, val in col_data.items():
                if pd.isna(val) or val == '' or val == 'nan':
                    continue
                try:
                    # Simulate your clean_value function
                    s = str(val).strip()
                    if '/' in s:
                        s = s.split('/')[0].strip()
                    import re
                    match = re.search(r'\d+(\.\d+)?', s)
                    if match:
                        int(round(float(match.group(0))))
                    else:
                        problems.append(f"❌ {col_name} Row {idx}: Cannot parse '{val}' as int")
                        print(f"  ⚠️  Row {idx}: '{val}' → FAILED")
                except Exception as e:
                    problems.append(f"❌ {col_name} Row {idx}: {val} → {e}")
                    print(f"  ⚠️  Row {idx}: '{val}' → ERROR: {e}")
        
        elif dtype == 'float':
            for idx, val in col_data.items():
                if pd.isna(val) or val == '' or val == 'nan':
                    continue
                try:
                    s = str(val).strip()
                    if '/' in s:
                        s = s.split('/')[0].strip()
                    import re
                    match = re.search(r'\d+(\.\d+)?', s)
                    if match:
                        float(match.group(0))
                    else:
                        problems.append(f"❌ {col_name} Row {idx}: Cannot parse '{val}' as float")
                        print(f"  ⚠️  Row {idx}: '{val}' → FAILED")
                except Exception as e:
                    problems.append(f"❌ {col_name} Row {idx}: {val} → {e}")
                    print(f"  ⚠️  Row {idx}: '{val}' → ERROR: {e}")
        
        elif dtype == 'bool':
            for idx, val in col_data.items():
                if pd.isna(val) or val == '' or val == 'nan':
                    continue
                val_str = str(val).strip().lower()
                if val_str not in ['yes', 'no', 'true', 'false', '1', '0', 't', 'f']:
                    problems.append(f"❌ {col_name} Row {idx}: '{val}' not a valid boolean")
                    print(f"  ⚠️  Row {idx}: '{val}' → INVALID BOOLEAN")
        
        elif dtype == 'str':
            # Check string length constraints
            if 'VARCHAR' in sql_type:
                max_len = int(sql_type.split('(')[1].split(')')[0])
                for idx, val in col_data.items():
                    if pd.isna(val) or val == '' or val == 'nan':
                        continue
                    if len(str(val)) > max_len:
                        problems.append(f"❌ {col_name} Row {idx}: '{val}' exceeds max length {max_len}")
                        print(f"  ⚠️  Row {idx}: '{val}' → TOO LONG (len={len(str(val))})")
        
        print(f"✅ {col_name} validation complete")
    
    # Summary
    print(f"\n\n{'='*60}")
    print("VALIDATION SUMMARY")
    print(f"{'='*60}")
    
    if problems:
        print(f"\n❌ Found {len(problems)} problems:\n")
        for p in problems:
            print(f"  {p}")
    else:
        print("\n✅ All columns validated successfully!")
    
    return problems


# Load and validate
df = pd.read_csv(os.path.join('data', 'monitors_clean2.csv'))
problems = validate_monitor_data(df)

# Focus on first few rows if too many errors
if len(problems) > 50:
    print("\n⚠️  Too many errors! Testing first 10 rows only...")
    df_test = df.head(10)
    problems_test = validate_monitor_data(df_test)


Validating: Size (inch) (Expected: DECIMAL(4, 1))
  ⚠️  Row 13: 'Nano-Texture Glass and Tilt Adjustable / Nano-texture Glass Tilt and Height-Adjustable / Nano-texture Glass and VESA Mount Adapter / Standard Glass Tilt and Height-Adjustable / Standard Glass and Tilt Adjustable / Standard Glass and VESA Mount Adapter' → FAILED
  ⚠️  Row 183: 'Blue / Green / Pink / White' → FAILED
✅ Size (inch) validation complete

Validating: Curve Radius (Expected: VARCHAR(20))
✅ Curve Radius validation complete

Validating: Wall Mount (Expected: VARCHAR(50))
✅ Wall Mount validation complete

Validating: Borders Size (cm) (Expected: DECIMAL(4, 2))
✅ Borders Size (cm) validation complete

Validating: Brightness (Expected: DECIMAL(3, 1))
✅ Brightness validation complete

Validating: Response Time (Expected: DECIMAL(3, 1))
✅ Response Time validation complete

Validating: HDR Picture (Expected: DECIMAL(3, 1))
✅ HDR Picture validation complete

Validating: SDR Picture (Expected: DECIMAL(3, 1))
✅ SDR Picture

In [77]:
from perplexity import Perplexity
from dotenv import load_dotenv
import os

load_dotenv()

client = Perplexity(api_key=os.environ.get("PERPLEXITY_KEY"))

search = client.search.create(
    query=[
      "what is RGB in monitors"
    ]
)

# for result in search.results:
#     print(f"{result.title}: {result.url}")
for result in search.results:
    print(result.snippet)
    break

The **RGB color model** is an additive color model in which the red, green, and blue primary colors of light are added together in various ways to reproduce a broad array of colors. The name of the model comes from the initials of the three additive primary colors, red, green, and blue.

The main purpose of the RGB color model is for the sensing, representation, and display of images in electronic systems, such as televisions and computers, though it has also been used in conventional photography and colored lighting. Before the electronic age, the RGB color model already had a solid theory behind it, based in human perception of colors.... RGB is a *device-dependent* color model: different devices detect or reproduce a given RGB value differently, since the color elements (such as phosphors or dyes) and their response to the individual red, green, and blue levels vary from manufacturer to manufacturer, or even in the same device over time. Thus an RGB value does not define the same *c

## Create VectorDB

In [78]:
from dotenv import load_dotenv
import chromadb
load_dotenv()

chroma_client = chromadb.PersistentClient(path="vectorDB")

# Create (or get) a collection
chroma_client.delete_collection("electronics")

collection = chroma_client.get_or_create_collection(name="electronics")

In [79]:
from openai import OpenAI

# Initialize OpenAI client
openai_client = OpenAI()

def get_embeddings(texts, model="text-embedding-3-small"):
    response = openai_client.embeddings.create(
        input=texts,
        model=model
    )
    return [d.embedding for d in response.data]

### Create Emebeddings for References

In [84]:
import json

with open(os.path.join("guide", "Guides-2.txt"), "r") as f:
    texts = f.readlines()
    f.close()

category = None
spec = None
text = []
json_data = []
for line in texts:
    if "**" in line:
        category = line.replace("*", "").strip()
    elif "*" in line:
        if category and spec:
            json_data.append({
                "type": "spec",
                "category_name": category.lower() if "key" not in category.lower() else "keyboards",
                "spec_name": spec.lower(),
                "text": "\n".join(text)
            })
        spec = line.replace("*", "").strip()
        text = []
    else:
        text.append(line)

if spec and text:
    json_data.append({
        "category_name": category.lower(),
        "spec_name": "spec",
        "text": " ".join(text)
    })

with open(os.path.join("guide", "texts.json"), "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=2)

In [87]:
import os
import json

with open(os.path.join("guide", "texts.json"), "r") as f:
    data = json.load(f)

ref_documents = [item["text"] for item in data]
ref_metadatas = [
    {
        "type": "spec",
        "category_name": item["category_name"],
        "spec_name": item["spec_name"]
    }
    for item in data
]
ref_ids = [f"doc_{i}" for i in range(len(data))]

In [88]:
ref_embeddings = get_embeddings(ref_documents)
len(ref_embeddings)

66

In [89]:
collection.add(
    embeddings=ref_embeddings,
    documents=ref_documents,
    metadatas=ref_metadatas,
    ids=ref_ids
)

### Creating Embeddings for Reviews

In [90]:
df = pd.read_csv("reviews.csv")

# Keep only the desired columns (in case CSV has more)
cols = ["product_id", "rating", "source", "review_title", "review_text"]
df = df[cols].dropna()

# Write to JSONL
output_path = os.path.join("guide", "reviews.jsonl")
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json_record = row.to_dict()
        f.write(json.dumps(json_record, ensure_ascii=False) + "\n")

print("JSONL file created:", output_path)

JSONL file created: guide/reviews.jsonl


In [91]:
def load_reviews_and_embed(jsonl_path: str):
    documents = []
    metadatas = []
    ids = []

    # Read JSONL file line by line
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            item = json.loads(line)

            # Combine review_title + review_text
            combined_text = ""
            if item.get("review_title"):
                combined_text += item["review_title"].strip() + ". "
            if item.get("review_text"):
                combined_text += item["review_text"].strip()

            if combined_text in [".", "", None]:
                continue

            documents.append(combined_text)

            # Build metadata
            metadata = {
                "product_id": item.get("product_id"),
                "rating": item.get("rating"),
                "source": item.get("source"),
                "type": "reviews"
            }
            metadatas.append(metadata)

            # Generate a unique ID
            ids.append(f"review_{i}")

    return documents, metadatas, ids


rev_documents, rev_metadatas, _ = load_reviews_and_embed(output_path)

In [92]:
from tqdm import tqdm

batch_size = 20
rev_embeddings = []

for i in tqdm(range(0, len(rev_documents), batch_size)):
    batch = rev_documents[i : i + batch_size]

    try:
        batch_embeddings = get_embeddings(batch)
        rev_embeddings.extend(batch_embeddings)
    except Exception as e:
        print("Error on batch starting at index:", i)
        print("Batch content:", batch)
        print("Exception:", e)
        break

100%|██████████| 125/125 [01:07<00:00,  1.86it/s]


In [97]:
prev_ids = collection.get()["ids"]
rev_ids = [f"doc_{i}" for i in range(len(prev_ids), len(prev_ids) + len(rev_documents))]

In [100]:
len(rev_ids), len(rev_embeddings), len(rev_documents), len(rev_metadatas)

(2499, 2499, 2499, 2499)

In [99]:
collection.add(
    embeddings=rev_embeddings,
    documents=rev_documents,
    metadatas=rev_metadatas,
    ids=rev_ids
)